In [3]:
import sys
import os
import time
import pprint

import boto3
# importing forecast notebook utility from notebooks/common directory
#sys.path.insert( 0, os.path.abspath("../../common") )
import util

pp = pprint.PrettyPrinter(indent=2)  # Better display for dictionaries

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rc('figure',figsize=(12,6))
import seaborn as sns

In [5]:
import warnings
warnings.filterwarnings("ignore")


In [6]:
train = pd.read_csv('train.csv', parse_dates=[0])
test = pd.read_csv('test.csv', parse_dates=[0])

In [95]:
train.date.nunique()

1826

In [94]:
test.date.nunique()

array(['2018-01-01', '2018-01-02', '2018-01-03', '2018-01-04',
       '2018-01-05', '2018-01-06', '2018-01-07', '2018-01-08',
       '2018-01-09', '2018-01-10', '2018-01-11', '2018-01-12',
       '2018-01-13', '2018-01-14', '2018-01-15', '2018-01-16',
       '2018-01-17', '2018-01-18', '2018-01-19', '2018-01-20',
       '2018-01-21', '2018-01-22', '2018-01-23', '2018-01-24',
       '2018-01-25', '2018-01-26', '2018-01-27', '2018-01-28',
       '2018-01-29', '2018-01-30', '2018-01-31', '2018-02-01',
       '2018-02-02', '2018-02-03', '2018-02-04', '2018-02-05',
       '2018-02-06', '2018-02-07', '2018-02-08', '2018-02-09',
       '2018-02-10', '2018-02-11', '2018-02-12', '2018-02-13',
       '2018-02-14', '2018-02-15', '2018-02-16', '2018-02-17',
       '2018-02-18', '2018-02-19', '2018-02-20', '2018-02-21',
       '2018-02-22', '2018-02-23', '2018-02-24', '2018-02-25',
       '2018-02-26', '2018-02-27', '2018-02-28', '2018-03-01',
       '2018-03-02', '2018-03-03', '2018-03-04', '2018-

In [ ]:
train = train.sort_values(by='date')

In [57]:
print(train.shape)
train.head()

(913000, 4)


,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [7]:
def print_error(test, value_column, predictions,date_column='',date_index=False):
    if not date_index:
        test.index = test[date_column]
    test = test[[value_column]]

    test['predictions'] = predictions
        
    test.plot(legend=True, figsize=(12,6))
    
    error_mse = mse(test[value_column], predictions)
    error_rmse = rmse(test[value_column], predictions)
    print(f'MSE: {error_mse:.2f}, RMSE: {error_rmse:.2f}')
    test_mean = test.mean()
    predictions_mean = predictions.mean()
    print(f'But the mean should be somewhat similar -> test_mean: {test_mean}, predictions mean: {predictions_mean}')

In [71]:
train.resample('D', on='date').sum()['sales']

date
2013-01-01    13696
2013-01-02    13678
2013-01-03    14488
2013-01-04    15677
2013-01-05    16237
              ...  
2017-12-27    20378
2017-12-28    21885
2017-12-29    23535
2017-12-30    24988
2017-12-31    26420
Freq: D, Name: sales, Length: 1826, dtype: int64

In [72]:
train_to_forecast = train.drop('store', axis=1)

In [85]:
train_to_forecast = train_to_forecast.groupby(['date', 'item']).sum().reset_index()

In [86]:
x = train_to_forecast[:-31]
y = train_to_forecast[-31:]

In [87]:
x.head()

,date,item,sales
0,2013-01-01,1,133
1,2013-01-01,2,327
2,2013-01-01,3,172
3,2013-01-01,4,102
4,2013-01-01,5,83


In [88]:
x.to_csv('forecast_train.csv', header=False, index=False)
y.to_csv('forecast_test.csv', header=False, index=False)

In [89]:
x.dtypes

date     datetime64[ns]
item              int64
sales             int64
dtype: object

In [96]:
x.head()

,date,item,sales
0,2013-01-01,1,133
1,2013-01-01,2,327
2,2013-01-01,3,172
3,2013-01-01,4,102
4,2013-01-01,5,83


In [92]:
xt = train[:-31]
yt = train[-31:]
xt.to_csv('forecast_full.csv', header=False, index=False)

In [97]:
test.head()

,id,date,store,item
0,0,2018-01-01,1,1
1,1,2018-01-02,1,1
2,2,2018-01-03,1,1
3,3,2018-01-04,1,1
4,4,2018-01-05,1,1


In [99]:
test.tail()

,id,date,store,item
44995,44995,2018-03-27,10,50
44996,44996,2018-03-28,10,50
44997,44997,2018-03-29,10,50
44998,44998,2018-03-30,10,50
44999,44999,2018-03-31,10,50


In [101]:
test.item.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50])

In [8]:
fr0 = pd.read_csv('forecast_export_item1_2021-04-08T19-33-53Z_part0.csv')
fr1 = pd.read_csv('forecast_export_item1_2021-04-08T19-33-53Z_part1.csv')
fr2 = pd.read_csv('forecast_export_item1_2021-04-08T19-33-53Z_part2.csv')
fr3 = pd.read_csv('forecast_export_item1_2021-04-08T19-33-53Z_part3.csv')

In [117]:
fr0.head()

,item_id,date,store,p10,p50,p90
0,24,2018-01-01T00:00:00Z,1,23.710733,34.701153,43.378487
1,24,2018-01-02T00:00:00Z,1,33.661163,43.381706,52.266064
2,24,2018-01-03T00:00:00Z,1,34.368095,42.373062,51.194790
3,24,2018-01-04T00:00:00Z,1,36.460640,46.856777,56.094807
4,24,2018-01-05T00:00:00Z,1,39.956226,50.281727,59.517036


,item_id,date,store,p10,p50,p90
0,43,2018-01-01T00:00:00Z,7,15.035352,23.972351,31.041718
1,43,2018-01-02T00:00:00Z,7,20.536219,28.033861,34.835148
2,43,2018-01-03T00:00:00Z,7,21.629093,27.995485,34.892693
3,43,2018-01-04T00:00:00Z,7,21.818283,30.309904,38.103474
4,43,2018-01-05T00:00:00Z,7,24.421043,32.224472,39.394394


In [9]:
fr0.shape[0] + fr1.shape[0] + fr2.shape[0] + fr3.shape[0] , test.shape

(45000, (45000, 4))

In [24]:
forecast = pd.concat([fr0,fr1,fr2,fr3], axis=0)

In [25]:
forecast.head()

,item_id,date,store,p10,p50,p90
0,24,2018-01-01T00:00:00Z,1,23.710733,34.701153,43.378487
1,24,2018-01-02T00:00:00Z,1,33.661163,43.381706,52.266064
2,24,2018-01-03T00:00:00Z,1,34.368095,42.373062,51.194790
3,24,2018-01-04T00:00:00Z,1,36.460640,46.856777,56.094807
4,24,2018-01-05T00:00:00Z,1,39.956226,50.281727,59.517036


In [31]:
df_pred = forecast.copy()

In [32]:
df_pred['id'] = df_pred['item_id']

In [34]:
df_pred['sales10'] = df_pred['p10']
df_pred['sales'] = df_pred['p50']
df_pred['sales90'] = df_pred['p90']

In [35]:
df_pred.drop(['item_id','p10','p50','p90'], axis=1,inplace=True)

In [36]:
df_pred.head()

,date,store,id,sales10,sales,sales90
0,2018-01-01T00:00:00Z,1,24,23.710733,34.701153,43.378487
1,2018-01-02T00:00:00Z,1,24,33.661163,43.381706,52.266064
2,2018-01-03T00:00:00Z,1,24,34.368095,42.373062,51.194790
3,2018-01-04T00:00:00Z,1,24,36.460640,46.856777,56.094807
4,2018-01-05T00:00:00Z,1,24,39.956226,50.281727,59.517036


In [37]:
df_pred.dtypes

date        object
store        int64
id           int64
sales10    float64
sales      float64
sales90    float64
dtype: object

In [38]:
from datetime import datetime

In [40]:
df_pred['date'] = pd.to_datetime(df_pred['date'])
df_pred['date'] = df_pred['date'].apply(lambda x: x.strftime('%Y-%m-%d'))
df_pred.head()

,date,store,id,sales10,sales,sales90
0,2018-01-01,1,24,23.710733,34.701153,43.378487
1,2018-01-02,1,24,33.661163,43.381706,52.266064
2,2018-01-03,1,24,34.368095,42.373062,51.194790
3,2018-01-04,1,24,36.460640,46.856777,56.094807
4,2018-01-05,1,24,39.956226,50.281727,59.517036


In [47]:
df_pred.rename(columns={'id': 'item_id'}, inplace=True)

In [51]:
df_sorted = df_pred.sort_values(by=['item_id','store','date'])

In [53]:
ids = list(range(45000))
sales = df_sorted.sales
data = dict(id=ids, sales=sales)
df_submission = pd.DataFrame(data)

In [54]:
df_submission.head()

,id,sales
3870,0,12.487864
3871,1,17.644295
3872,2,15.187359
3873,3,15.710437
3874,4,18.179716


In [56]:
df_submission.to_csv('submission.csv', index=False, header=True)

In [55]:
!pip install kaggle